In [8]:
import numpy as np
import os
import pathlib
import random
import torch
import torchgeometry
import cv2
from dlclive import DLCLive, Processor



from torch.utils.data import DataLoader

# potential mass parametrizations
from differentiable_robot_model.rigid_body_params import (
    UnconstrainedScalar,
    PositiveScalar,
    UnconstrainedTensor,
)

# potential inertia matrix parametrizations
from differentiable_robot_model.rigid_body_params import (
    CovParameterized3DInertiaMatrixNet,
    Symm3DInertiaMatrixNet,
    SymmPosDef3DInertiaMatrixNet,
    TriangParam3DInertiaMatrixNet,
)

from differentiable_robot_model.robot_model import (
    DifferentiableRobotModel,
    DifferentiableKUKAiiwa,
)
from differentiable_robot_model.data_utils import (
    generate_sine_motion_forward_dynamics_data,
)
import diff_robot_data

torch.set_printoptions(precision=3, sci_mode=False)

random.seed(0)
np.random.seed(1)
torch.manual_seed(0)


In [4]:
class NMSELoss(torch.nn.Module):
    def __init__(self, var):
        super(NMSELoss, self).__init__()
        self.var = var

    def forward(self, yp, yt):
        err = (yp - yt) ** 2
        werr = err / self.var
        return werr.mean()


urdf_path = os.path.join("a1.urdf")
device = "cpu"
learnable_robot_model = DifferentiableRobotModel(
    urdf_path, "A1", device=device
)

learnable_robot_model.make_link_param_learnable(
    "FR_hip", "trans", UnconstrainedTensor(dim1=1, dim2=3)
)
learnable_robot_model.make_link_param_learnable(
        "FR_thigh_shoulder", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FR_thigh", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FR_calf", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FR_foot", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FL_hip", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FL_thigh_shoulder", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FL_thigh", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FL_calf", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FL_foot", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RR_hip", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RR_thigh_shoulder", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RR_thigh", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RR_calf", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RR_foot", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RL_hip", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RL_thigh_shoulder", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RL_thigh", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RL_calf", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RL_foot", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )

# learnable_robot_model.print_learnable_params()
joint_angles = torch.rand((1, 12), requires_grad=True)
learnable_robot_model.compute_forward_kinematics(joint_angles,"FR_foot")

Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FR_hip_tran']/actuator[@name='FR_hip_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FR_thigh_tran']/actuator[@name='FR_thigh_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FR_calf_tran']/actuator[@name='FR_calf_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FL_hip_tran']/actuator[@name='FL_hip_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FL_thigh_tran']/actuator[@name='FL_thigh_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FL_calf_tran']/actuator[@name='FL_calf_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='RR_hip_tran']/actuator[@name='RR_hip_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='RR_thigh_tran']/actuator[@name='RR_thigh_motor']
Unknown tag "hardwareInterface" 

(tensor([[-0.431,  0.199,  0.189]], grad_fn=<AddBackward0>),
 tensor([[0.268, 0.539, 0.186, 0.776]], grad_fn=<CopySlices>))

In [5]:
keypoints = """End of Neck
Shoulder
FL_Knee
FL_Ankle
FL_White_TapeTop
FL_White_TapeBot
FR_Knee
FR_Ankle
FL_Red_TapeTop
FL_Red_TapeBot
End of Tail
Hip
BL_Knee
BL_Ankle
BL_Red_TapeTop
BL_Red_TapeBot
BR_Knee
BR_Ankle
BR_Red_TapeTop
BR_Red_TapeBot""".split("\n")
training_keypoints = ['FL_Ankle','FL_Knee','BL_Ankle','BL_Knee']
indices_keypoints_training = [keypoints.index(val) for val in training_keypoints]

base_dir = pathlib.Path.cwd()
img_path = base_dir / "HorseInferenceFiles/img0088.png"
dlc_model_path = base_dir/"HorseInferenceFiles/DLC_HorseProject1_efficientnet-b0_iteration-0_shuffle-1"
image = cv2.imread(str(img_path))
dlc_proc = Processor()
dlc_live = DLCLive(model_path=str(dlc_model_path), processor=dlc_proc)
dlc_live.init_inference(image)
img_keypoints = dlc_live.get_pose(image)

training_data = img_keypoints[indices_keypoints_training]
training_data = training_data[:,0:2]
training_data

2023-06-03 09:43:43.184513: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-03 09:43:43.224776: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


array([[249.62645, 216.09793],
       [275.79907, 244.88426],
       [349.00183, 210.68068],
       [340.4294 , 246.94559]], dtype=float32)

In [14]:
intrinsic = torch.eye(4,requires_grad=True)
extrinsic = torch.ones((4,4), requires_grad=True)
intrinsic, extrinsic = intrinsic.unsqueeze(0), extrinsic.unsqueeze(0)

imageHeight,imageWidth = torch.tensor(image.shape[0]),torch.tensor(image.shape[1])
imageHeight = imageHeight.unsqueeze(0)
imageWidth = imageWidth.unsqueeze(0)

cameraModel = torchgeometry.PinholeCamera(intrinsic, extrinsic, imageHeight, torch.tensor([1,]))
pixel_coords = torch.rand((1,training_data.shape[0], 1, 3), requires_grad=True)

#Projection is in 3D space where the relevant points on the robot thinks it is
projection = torch.cat((learnable_robot_model.compute_forward_kinematics(joint_angles, "FL_foot")[0],\
learnable_robot_model.compute_forward_kinematics(joint_angles, "FL_calf")[0],\
learnable_robot_model.compute_forward_kinematics(joint_angles, "RL_foot")[0],\
learnable_robot_model.compute_forward_kinematics(joint_angles, "RL_calf")[0]))
projection = projection.unsqueeze(0).unsqueeze(0)

depth = torch.ones((1,1,1,4))

cameraEstimate = torchgeometry.pixel2cam(depth, cameraModel.intrinsics_inverse(), projection)
cameraEstimate = cameraEstimate.squeeze(0).squeeze(0)[:,0:2]
cameraEstimate, cameraEstimate.shape

(tensor([[-0.155,  0.087],
         [-0.015,  0.018],
         [-0.203, -0.239],
         [-0.059, -0.295]], grad_fn=<SliceBackward0>),
 torch.Size([4, 2]))

In [15]:
cameraLoss = torch.sum(torch.sqrt(torch.sum(torch.pow(torch.from_numpy(training_data)-cameraEstimate, 2), dim = 1)))
cameraLoss

tensor(1527.802, grad_fn=<SumBackward0>)

In [22]:
intrinsic.retain_grad()
optimizer = torch.optim.Adam([joint_angles]+ list(intrinsic) + list(learnable_robot_model.parameters()), lr=1e-2)
for _ in range(1000):
    optimizer.zero_grad()
    
    #Where the Robot Thinks It Is
    projection = torch.cat((learnable_robot_model.compute_forward_kinematics(joint_angles, "FL_foot")[0],\
    learnable_robot_model.compute_forward_kinematics(joint_angles, "FL_calf")[0],\
    learnable_robot_model.compute_forward_kinematics(joint_angles, "RR_foot")[0],\
    learnable_robot_model.compute_forward_kinematics(joint_angles, "RR_calf")[0]))
    projection = projection.unsqueeze(0).unsqueeze(0)

    #Where Robot Is In Pixel Space
    cameraModel = torchgeometry.PinholeCamera(intrinsic, extrinsic, imageHeight, torch.tensor([1,]))
    cameraEstimate = torchgeometry.pixel2cam(depth, cameraModel.intrinsics_inverse(), projection)
    cameraEstimate = cameraEstimate.squeeze(0).squeeze(0)[:,0:2]

    #Loss Between Where Robot/Horse Should Be and Where It Is

    loss = torch.sum(torch.sqrt(torch.sum(torch.pow(torch.from_numpy(training_data)-cameraEstimate, 2))))
    print(loss)
    loss.backward()
    optimizer.step()


ValueError: can't optimize a non-leaf Tensor